In [ ]:
import math

def f0(x):
    """Полином степени 0: f(x) = 1"""
    return 1

def f1(x):
    """Полином степени 1: f(x) = 2x + 3"""
    return 2*x + 3

def f2(x):
    """Полином степени 2: f(x) = x^2 - 4x + 5"""
    return x**2 - 4*x + 5

def f3(x):
    """Полином степени 3: f(x) = x^3 - 2x^2 + x - 7"""
    return x**3 - 2*x**2 + x - 7

def f4(x):
    """Не полином: f(x) = sin(x)"""
    return math.sin(x)

# ПЕРВООБРАЗНЫЕ
def exact_integral(f_num, a, b):
    """Точное значение интеграла от a до b для выбранной функции"""
    if f_num == 0:
        return b - a
    elif f_num == 1:
        return (b**2 - a**2) + 3*(b - a)  # ∫(2x+3)dx = x^2 + 3x
    elif f_num == 2:
        # ∫(x^2 - 4x + 5)dx = (x^3/3) - 2x^2 + 5x
        return (b**3/3 - 2*b**2 + 5*b) - (a**3/3 - 2*a**2 + 5*a)
    elif f_num == 3:
        # ∫(x^3 - 2x^2 + x - 7)dx = x^4/4 - (2/3)x^3 + x^2/2 - 7x
        return (b**4/4 - (2/3)*b**3 + b**2/2 - 7*b) - \
               (a**4/4 - (2/3)*a**3 + a**2/2 - 7*a)
    elif f_num == 4:
        return -math.cos(b) + math.cos(a)
    else:
        return None

#СОСТАВНЫЕ КФ
def composite_left(f, a, b, m):
    """СКФ левых прямоугольников"""
    h = (b - a) / m
    s = 0.0
    for j in range(m):
        s += f(a + j*h)
    return h * s

def composite_right(f, a, b, m):
    """СКФ правых прямоугольников"""
    h = (b - a) / m
    s = 0.0
    for j in range(1, m+1):
        s += f(a + j*h)
    return h * s

def composite_mid(f, a, b, m):
    """СКФ средних прямоугольников"""
    h = (b - a) / m
    s = 0.0
    for j in range(m):
        s += f(a + (j + 0.5)*h)
    return h * s

def composite_trapezoidal(f, a, b, m):
    """СКФ трапеций"""
    h = (b - a) / m
    s = 0.5 * (f(a) + f(b))
    for j in range(1, m):
        s += f(a + j*h)
    return h * s

def composite_simpson(f, a, b, m):
    """СКФ Симпсона (m должно быть чётным)"""
    if m % 2 != 0:
        m += 1  # делаем чётным
    h = (b - a) / m
    s = f(a) + f(b)
    # чётные узлы
    for j in range(2, m, 2):
        s += 2 * f(a + j*h)
    # нечётные узлы
    for j in range(1, m, 2):
        s += 4 * f(a + j*h)
    return h * s / 3

# РУНГЕ-РОМБЕРГ
def runge_romberg(J_h, J_hl, l, r):
    """
    Уточнение по Рунге-Ромбергу
    J_h  - значение с шагом h
    J_hl - значение с шагом h/l
    l    - во сколько раз уменьшили шаг
    r    - порядок малости погрешности (r = АСТ+1 для СКФ)
    """
    return (l**r * J_hl - J_h) / (l**r - 1)

# ТАБЛИЦА РЕЗОВ
def print_results_table(methods_results, exact, m, l):
    """
    methods_results: список кортежей (name, J_h, J_hl, r)
    exact: точное значение
    m: исходное число разбиений
    l: коэффициент уменьшения шага
    """
    print("\n" + "="*120)
    print(f"{'Название СКФ':<25} | {'J(h)':<12} | {'|J-J(h)|':<12} | {'J(h/l)':<12} | {'|J-J(h/l)|':<12} | {'J_R':<12} | {'|J-J_R|':<12}")
    print("-"*120)
    
    for name, J_h, J_hl, r in methods_results:
        abs_err_h = abs(exact - J_h)
        abs_err_hl = abs(exact - J_hl)
        J_R = runge_romberg(J_h, J_hl, l, r)
        abs_err_R = abs(exact - J_R)
        
        print(f"{name:<25} | {J_h:<12.8f} | {abs_err_h:<12.8f} | {J_hl:<12.8f} | {abs_err_hl:<12.8f} | {J_R:<12.8f} | {abs_err_R:<12.8f}")


def main():
    functions = [f0, f1, f2, f3, f4]
    func_names = [
        "f0(x) = 1",
        "f1(x) = 2x + 3",
        "f2(x) = x^2 - 4x + 5",
        "f3(x) = x^3 - 2x^2 + x - 7",
        "f4(x) = sin(x)"
    ]
    
    # Порядки точности по Рунге для КФ
    orders = {
        "СКФ левых прямоугольников": 1,      # АСТ=0
        "СКФ правых прямоугольников": 1,     # АСТ=0
        "СКФ средних прямоугольников": 2,    # АСТ=1
        "СКФ трапеций": 2,                   # АСТ=1
        "СКФ Симпсона": 4                    # АСТ=3
    }
    
    while True:
        print("\n" + "="*60)
        print("Выберите функцию для интегрирования:")
        for i, name in enumerate(func_names):
            print(f"{i}: {name}")
        print("5: Выход")
        
        try:
            choice = int(input("Ваш выбор: "))
        except:
            print("Ошибка ввода")
            continue
        
        if choice == 5:
            print("Выход из программы.")
            break
        if choice < 0 or choice > 4:
            print("Неверный выбор.")
            continue
        
        try:
            a = float(input("Введите левый предел интегрирования a: "))
            b = float(input("Введите правый предел интегрирования b: "))
            m = int(input("Введите число разбиений m: "))
        except:
            print("Ошибка ввода чисел.")
            continue
        
        if a > b:
            a, b = b, a
        
        f = functions[choice]
        exact = exact_integral(choice, a, b)
        h = (b - a) / m
        
        print(f"\nТочное значение интеграла J = {exact:.10f}")
        print(f"Параметры: A={a}, B={b}, m={m}, h={h:.6f}")
        
        # Вычисление для каждого метода
        methods = [
            ("СКФ левых прямоугольников", composite_left),
            ("СКФ правых прямоугольников", composite_right),
            ("СКФ средних прямоугольников", composite_mid),
            ("СКФ трапеций", composite_trapezoidal),
            ("СКФ Симпсона", composite_simpson)
        ]
        
        print("\n" + "-"*60)
        print("Результаты для шага h:")
        print(f"{'Метод':<30} | {'Приближение':<15} | {'Абс. погр.':<15} | {'Отн. погр. (%)':<15}")
        print("-"*60)
        
        results_h = {}
        for name, method in methods:
            approx = method(f, a, b, m)
            abs_err = abs(exact - approx)
            rel_err = abs_err / abs(exact) * 100 if exact != 0 else float('inf')
            results_h[name] = approx
            print(f"{name:<30} | {approx:<15.10f} | {abs_err:<15.10f} | {rel_err:<15.10f}")
        
        # Уточнение по Рунге-Ромбергу
        print("\n" + "="*60)
        print("ЭТАП УТОЧНЕНИЯ ПО РУНГЕ-РОМБЕРГУ")
        
        try:
            l = int(input("Введите коэффициент уменьшения шага l: "))
        except:
            print("Ошибка ввода, используем l=2")
            l = 2
        
        m_new = m * l
        h_new = (b - a) / m_new
        
        print(f"\nНовые параметры: m' = {m_new}, h' = {h_new:.6f}")
        print("\nРезультаты для шага h/l:")
        print(f"{'Метод':<30} | {'Приближение':<15} | {'Абс. погр.':<15} | {'Отн. погр. (%)':<15}")
        print("-"*60)
        
        results_hl = {}
        for name, method in methods:
            approx = method(f, a, b, m_new)
            abs_err = abs(exact - approx)
            rel_err = abs_err / abs(exact) * 100 if exact != 0 else float('inf')
            results_hl[name] = approx
            print(f"{name:<30} | {approx:<15.10f} | {abs_err:<15.10f} | {rel_err:<15.10f}")
        
        # Формируем данные для таблицы
        table_data = []
        for name, method in methods:
            J_h = results_h[name]
            J_hl = results_hl[name]
            r = orders[name]
            table_data.append((name, J_h, J_hl, r))
        
        # Выводим таблицу
        print_results_table(table_data, exact, m, l)
        
        print("\n" + "="*60)
        print("АНАЛИЗ РЕЗУЛЬТАТОВ:")
        
        # 1. Проверка на полиномах
        if choice == 0:  # константа
            print("- Все СКФ должны быть точны (АСТ ≥ 0) → погрешности должны быть ~0")
        elif choice == 1:  # степень 1
            print("- СКФ средних прямоугольников и трапеций должны быть точны (АСТ=1)")
            print("- СКФ Симпсона тоже точна (АСТ=3 ≥ 1)")
        elif choice == 2:  # степень 2
            print("- Только СКФ Симпсона должна быть точна (АСТ=3 ≥ 2)")
        elif choice == 3:  # степень 3
            print("- Только СКФ Симпсона должна быть точна (АСТ=3 ≥ 3)")
        elif choice == 4:  # sin(x)
            print("- Ни одна формула не обязана быть точной для не-полинома")
            print("- СКФ Симпсона должна давать наименьшую погрешность (порядок h^4)")
        
        # 2. Эффективность уточнения Рунге
        print("\nЭффективность уточнения по Рунге-Ромбергу:")
        for name, method in methods:
            J_h = results_h[name]
            J_hl = results_hl[name]
            r = orders[name]
            J_R = runge_romberg(J_h, J_hl, l, r)
            err_R = abs(exact - J_R)
            err_h = abs(exact - J_h)
            
            if err_h > 1e-15:
                improvement = (err_h - err_R) / err_h * 100
                print(f"{name}: уменьшение погрешности на {improvement:.2f}%")
        
        # 3. Ответы на вопросы из задания
        print("\n" + "="*60)
        print("ОТВЕТЫ НА ВОПРОСЫ ИЗ ЗАДАНИЯ:")
        
        # Вопрос 1: Сколько значений функции участвует?
        print("\n1. Количество вычислений f(x) для m разбиений:")
        print(f"   - Левые/правые прямоугольники: {m} значений")
        print(f"   - Средние прямоугольники: {m} значений")
        print(f"   - Трапеции: {m+1} значений")
        print(f"   - Симпсона: {m+1} значений (m чётное)")
        
        # Вопрос 2: Почему средние прямоугольники и Симпсон точны для нечётной функции на симметричном отрезке?
        if choice == 4 and a == -b:  # sin(x) на [-a, a]
            print("\n2. Для нечётной функции на симметричном отрезке [-a, a]:")
            print("   - Интеграл точен = 0")
            print("   - Средние прямоугольники и Симпсон дают 0 из-за симметрии узлов")
            print("   - Поэтому они 'точны' даже если АСТ формально недостаточна")
        
        # Вопрос 3: Проверка "на прочность"
        print("\n3. Проверка 'на прочность':")
        print("   - При больших m программа должна оставаться устойчивой")
        print("   - Симпсон при умеренных m часто точнее, чем при очень больших")
        print("   - Причина: накопление ошибок округления при большом числе операций")

if __name__ == "__main__":
    main()
    


Выберите функцию для интегрирования:
0: f0(x) = 1
1: f1(x) = 2x + 3
2: f2(x) = x^2 - 4x + 5
3: f3(x) = x^3 - 2x^2 + x - 7
4: f4(x) = sin(x)
5: Выход


Ваш выбор:  1
Введите левый предел интегрирования a:  5
Введите правый предел интегрирования b:  10
Введите число разбиений m:  50



Точное значение интеграла J = 90.0000000000
Параметры: A=5.0, B=10.0, m=50, h=0.100000

------------------------------------------------------------
Результаты для шага h:
Метод                          | Приближение     | Абс. погр.      | Отн. погр. (%) 
------------------------------------------------------------
СКФ левых прямоугольников      | 89.5000000000   | 0.5000000000    | 0.5555555556   
СКФ правых прямоугольников     | 90.5000000000   | 0.5000000000    | 0.5555555556   
СКФ средних прямоугольников    | 90.0000000000   | 0.0000000000    | 0.0000000000   
СКФ трапеций                   | 90.0000000000   | 0.0000000000    | 0.0000000000   
СКФ Симпсона                   | 90.0000000000   | 0.0000000000    | 0.0000000000   

ЭТАП УТОЧНЕНИЯ ПО РУНГЕ-РОМБЕРГУ


Введите коэффициент уменьшения шага l:  2



Новые параметры: m' = 100, h' = 0.050000

Результаты для шага h/l:
Метод                          | Приближение     | Абс. погр.      | Отн. погр. (%) 
------------------------------------------------------------
СКФ левых прямоугольников      | 89.7500000000   | 0.2500000000    | 0.2777777778   
СКФ правых прямоугольников     | 90.2500000000   | 0.2500000000    | 0.2777777778   
СКФ средних прямоугольников    | 90.0000000000   | 0.0000000000    | 0.0000000000   
СКФ трапеций                   | 90.0000000000   | 0.0000000000    | 0.0000000000   
СКФ Симпсона                   | 90.0000000000   | 0.0000000000    | 0.0000000000   

Название СКФ              | J(h)         | |J-J(h)|     | J(h/l)       | |J-J(h/l)|   | J_R          | |J-J_R|     
------------------------------------------------------------------------------------------------------------------------
СКФ левых прямоугольников | 89.50000000  | 0.50000000   | 89.75000000  | 0.25000000   | 90.00000000  | 0.00000000  
СКФ пра

Ваш выбор:  3
Введите левый предел интегрирования a:  5
Введите правый предел интегрирования b:  10
Введите число разбиений m:  50



Точное значение интеграла J = 1762.9166666667
Параметры: A=5.0, B=10.0, m=50, h=0.100000

------------------------------------------------------------
Результаты для шага h:
Метод                          | Приближение     | Абс. погр.      | Отн. погр. (%) 
------------------------------------------------------------
СКФ левых прямоугольников      | 1726.5875000000 | 36.3291666667   | 2.0607421413   
СКФ правых прямоугольников     | 1799.5875000000 | 36.6708333333   | 2.0801229024   
СКФ средних прямоугольников    | 1762.8312500000 | 0.0854166667    | 0.0048451903   
СКФ трапеций                   | 1763.0875000000 | 0.1708333333    | 0.0096903805   
СКФ Симпсона                   | 1762.9166666667 | 0.0000000000    | 0.0000000000   

ЭТАП УТОЧНЕНИЯ ПО РУНГЕ-РОМБЕРГУ


Введите коэффициент уменьшения шага l:  2



Новые параметры: m' = 100, h' = 0.050000

Результаты для шага h/l:
Метод                          | Приближение     | Абс. погр.      | Отн. погр. (%) 
------------------------------------------------------------
СКФ левых прямоугольников      | 1744.7093750000 | 18.2072916667   | 1.0327936658   
СКФ правых прямоугольников     | 1781.2093750000 | 18.2927083333   | 1.0376388561   
СКФ средних прямоугольников    | 1762.8953125000 | 0.0213541667    | 0.0012112976   
СКФ трапеций                   | 1762.9593750000 | 0.0427083333    | 0.0024225951   
СКФ Симпсона                   | 1762.9166666667 | 0.0000000000    | 0.0000000000   

Название СКФ              | J(h)         | |J-J(h)|     | J(h/l)       | |J-J(h/l)|   | J_R          | |J-J_R|     
------------------------------------------------------------------------------------------------------------------------
СКФ левых прямоугольников | 1726.58750000 | 36.32916667  | 1744.70937500 | 18.20729167  | 1762.83125000 | 0.08541667  
СКФ 